<a href="https://colab.research.google.com/github/amengemeda/ISproject-2/blob/main/Amharic_Hate_Speech_detection_using_mBERT_(Trainer_API).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#installing the transformers package
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 13.0 MB/s 
     |████████████████████████████████| 182 kB 57.8 MB/s 
     |████████████████████████████████| 7.6 MB 57.6 MB/s 


In [2]:
#installing the datasets package
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 15.4 MB/s 
     |████████████████████████████████| 212 kB 66.5 MB/s 
     |████████████████████████████████| 115 kB 55.3 MB/s 
     |████████████████████████████████| 127 kB 36.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [3]:
#importing the datasets package
from datasets import Dataset
import datasets
#import load metric for model evaluation
from datasets import load_metric

In [4]:
#import numpy and pandas for mathematical computation and data manipulation respectively 
import numpy as np
import pandas as pd
#import drive package to connect this colab file with the drive where the data will be retrived from
from google.colab import drive
#import the pipeline of transformers
from transformers import pipeline
#import AutoTokenizer for tokenization purposes
from transformers import AutoTokenizer


In [5]:
#import torch
import torch
#import Data loader from torch
from torch.utils.data import DataLoader
#import an optimizer
from torch.optim import AdamW
#import tqdm for a progress bar
from tqdm.auto import tqdm

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
#mount google drive to access the dataset directly from the drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [8]:
#fetch the dataset from the drive
Labels=pd.read_csv('/content/drive/MyDrive/Machine Learning/Data/Amharic-Hate-Speech-Dataset/Labels.txt',header=None)
Posts=pd.read_csv('/content/drive/MyDrive/Machine Learning/Data/Amharic-Hate-Speech-Dataset/Posts.txt',header=None)

In [9]:
#naming the columns
Labels.columns = ["labels"]
Posts.columns = ["post"]

In [10]:
#encoding the classes into numerical data
Labels = Labels.replace(['Free', 'Free ','Hate'],[0,0,1]) 

In [11]:
#check the encoded label data
Labels.head(10)

,labels
0,0
1,0
2,0
3,0
4,1
5,0
6,0
7,0
8,1
9,0


In [12]:
#check the Amharic data
Posts.head(1000)

,post
0,አስቀድሜ ጥያቄዬ በጨዋነት በውስጥ መስመር እንዲደርስዎ አድርጌ ፍትህን ለ...
1,እነዚህን ወሳኝ ጉዳዮችን የሚያስፈፅም አካል እንዲቋቋምና ክትትል እንዲደ...
2,የአማራ ህዝብ በአእምሮ ክንፉ ያልበረረበት ጥበብና ፍልስፍና ያልከፈተው የ...
3,ከአማራ ህዝብ የሀገሪቱ ዘርፈ ብዙ እውቀት መንጭቶ የሞላበትከሙላቱም በመል...
4,ዛሬ በየትኛውም መለኪያ ይሁን መመዘኛ ኢትዮጵያዊነት የሚንፀባረቀው በአማራ...
...,...
995,መፈንቅለመንግስት ከሽፏልእንዴ የየክልሉ ህዝብ ምን ይጠብቃል ሆ ብሎ ሄዶ ...
996,ሰው በሩን እንደፈለገ መክፈት እንደፈለገ መዝጋት ይችላል የምን አውቅልሻ...
997,ክክክክክ የሱዳን ዜጋ ናችሁ እንዴ ባለ ነጠላ ጎጃሜ ሁላ
998,የምን ማለቃቀስ ነው ምድረ ፎካሪ ሁላ


In [13]:
#merge the datasets
Frames = [Labels, Posts]
Merged = pd.concat(Frames, axis=1)


In [14]:
#preview of merged data
Merged

,labels,post
0,0,አስቀድሜ ጥያቄዬ በጨዋነት በውስጥ መስመር እንዲደርስዎ አድርጌ ፍትህን ለ...
1,0,እነዚህን ወሳኝ ጉዳዮችን የሚያስፈፅም አካል እንዲቋቋምና ክትትል እንዲደ...
2,0,የአማራ ህዝብ በአእምሮ ክንፉ ያልበረረበት ጥበብና ፍልስፍና ያልከፈተው የ...
3,0,ከአማራ ህዝብ የሀገሪቱ ዘርፈ ብዙ እውቀት መንጭቶ የሞላበትከሙላቱም በመል...
4,1,ዛሬ በየትኛውም መለኪያ ይሁን መመዘኛ ኢትዮጵያዊነት የሚንፀባረቀው በአማራ...
...,...,...
29995,1,በአሉ የሁሉም ኢትዮጵያዊ ስላልሆነ በኦሮምኛው ቢለፋደድ ምን አገባን
29996,0,ተባረክ አብቹ ፈር ቀዳጅ ስለሆንህ መጋረጃው መቀደድ ስለጀመረ
29997,0,እስከ አሁን አንተ ብቻ ነው በ መፅሀፍ ያልቻልከው አንተም ታሪክ እ...
29998,1,ህገወጥት ጠቅላይ ሚንስትር ፅቤት የተፈቀደ ሆኖ ህዝብን እንዴት ህግ አክ...


In [15]:
train_val_df, test_dataset = train_test_split(Merged, test_size=0.20, random_state=42)
train_dataset, evaluation_dataset = train_test_split(train_val_df, test_size=0.10, random_state=42)
print('Training dataset shape: ', train_dataset.shape)
print('Validation dataset shape: ', evaluation_dataset.shape)
print('Testing dataset shape: ', test_dataset.shape)

Training dataset shape:  (21600, 2)
Validation dataset shape:  (2400, 2)
Testing dataset shape:  (6000, 2)


In [16]:
#Divide the dataset into train and test categories 
msk = np.random.rand(len(Merged)) < 0.8
train_dataset = Merged[msk]
test_dataset = Merged[~msk]

In [17]:
#convert format of the dataset to HuggingFace Dataset from Pandas DataFrame
test_dataset=Dataset.from_pandas(test_dataset)

In [18]:
#convert the format of the dataset to HuggingFace Dataset from Pandas DataFrame
train_dataset=Dataset.from_pandas(train_dataset)

In [19]:
#convert the format of the dataset to HuggingFace Dataset from Pandas DataFrame
evaluation_dataset=Dataset.from_pandas(evaluation_dataset)

In [20]:
#preview of the dataset after conversion
(test_dataset)

Dataset({
    features: ['labels', 'post', '__index_level_0__'],
    num_rows: 6046
})

In [21]:
#remove unnecessary column
test_dataset=test_dataset.remove_columns("__index_level_0__")
train_dataset=train_dataset.remove_columns("__index_level_0__")
evaluation_dataset=evaluation_dataset.remove_columns("__index_level_0__")

In [22]:
#combine the train and test dataset into one datset
main_dataset= datasets.DatasetDict({
    'train': train_dataset,
    'test': test_dataset,
    'evaluate': evaluation_dataset
})

In [23]:
#preview of the dataset after merging
main_dataset

DatasetDict({
    train: Dataset({
        features: ['labels', 'post'],
        num_rows: 23954
    })
    test: Dataset({
        features: ['labels', 'post'],
        num_rows: 6046
    })
    evaluate: Dataset({
        features: ['labels', 'post'],
        num_rows: 2400
    })
})

In [24]:
# training and testing data size
training_data_size = main_dataset['train'].num_rows
testing_data_size = main_dataset['test'].num_rows
evaluation_data_size = main_dataset['evaluate'].num_rows

In [25]:
#loading a tokenizer from the pretrained model
tokenizer = AutoTokenizer.from_pretrained("Davlan/bert-base-multilingual-cased-finetuned-amharic")

Downloading:   0%|          | 0.00/335 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.55M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [26]:
#Have a tokenizer function that uses the tokenizer 
def tokenize_function(data):
    return tokenizer(data["post"], padding="max_length", truncation=True)

In [27]:
#Tokenize all the data using the mapping functionality
tokenized_datasets = main_dataset.map(tokenize_function)


  0%|          | 0/23954 [00:00<?, ?ex/s]

  0%|          | 0/6046 [00:00<?, ?ex/s]

  0%|          | 0/2400 [00:00<?, ?ex/s]

In [28]:
#empty cache
torch.cuda.empty_cache()

In [29]:
#remove the posts column as it is no longer needed
tokenized_datasets = tokenized_datasets.remove_columns(["post"])

In [30]:
#changing the format of the tokenized dataset to torch
tokenized_datasets.set_format("torch")

In [31]:
#shuffeling and selecting the needed size of dataset for training and evaluating the model
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(training_data_size))
small_test_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(testing_data_size))
small_eval_dataset = tokenized_datasets["evaluate"].shuffle(seed=42).select(range(evaluation_data_size))

In [32]:
# preview of the shuffeled and selected evaluation dataset
small_eval_dataset

Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 2400
})

In [33]:
# preview of the shuffeled and selected training dataset
small_train_dataset

Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 23954
})

In [34]:
# preview of the shuffeled and selected testing dataset
small_test_dataset

Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 6046
})

In [35]:
#load the dataset using DataLoader
train_dataloader = DataLoader(small_train_dataset, shuffle=True, batch_size=4)
eval_dataloader = DataLoader(small_eval_dataset, batch_size=4)
test_dataloader = DataLoader(small_test_dataset, batch_size=4)

In [36]:
#Load auto mode classifier from the pretrained model 
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("Davlan/bert-base-multilingual-cased-finetuned-amharic", num_labels=2)
# model = AutoModelForSequenceClassification.from_pretrained("Davlan/", num_labels=2)

Downloading:   0%|          | 0.00/712M [00:00<?, ?B/s]

Some weights of the model checkpoint at Davlan/bert-base-multilingual-cased-finetuned-amharic were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model check

In [37]:
# #load an optimizer
# optimizer = AdamW(model.parameters(), lr=5e-5)
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [38]:
import numpy as np
from datasets import load_metric

metric = load_metric("f1","accuracy")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  after removing the cwd from sys.path.


In [39]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [40]:
from transformers import TrainingArguments, Trainer

In [41]:
from transformers import EarlyStoppingCallback, IntervalStrategy

In [42]:
# training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch", num_train_epochs=2)

In [52]:
training_args = TrainingArguments(
   f"training_with_callbacks",
   evaluation_strategy = IntervalStrategy.STEPS, # "steps"
   warmup_steps=500,                # number of warmup steps for learning rate  
   save_steps=2000,
   eval_steps = 2000, # Evaluation and Save happens every 50 steps
   save_total_limit = 3, # Only last 5 models are saved. Older ones are deleted.
   learning_rate=1e-5,
   per_device_train_batch_size=4,
   per_device_eval_batch_size=4,
   num_train_epochs=5,
   weight_decay=0.01,
   push_to_hub=False,
   metric_for_best_model = 'f1',
   load_best_model_at_end=True)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [53]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=10)],
)

In [54]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 23954
  Num Epochs = 5
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 29945
  Number of trainable parameters = 177854978


Step,Training Loss,Validation Loss,F1
2000,0.688000,0.653254,0.661533
4000,0.670400,0.668725,0.562652
6000,0.656000,0.630557,0.612436
8000,0.627900,0.642800,0.664308
10000,0.610300,0.559548,0.738499
12000,0.614600,0.533678,0.764143
14000,0.589100,0.580401,0.767509
16000,0.581400,0.488746,0.811239
18000,0.571000,0.519666,0.826466
20000,0.574200,0.541039,0.840800


***** Running Evaluation *****
  Num examples = 2400
  Batch size = 4
Saving model checkpoint to training_with_callbacks/checkpoint-2000
Configuration saved in training_with_callbacks/checkpoint-2000/config.json
Model weights saved in training_with_callbacks/checkpoint-2000/pytorch_model.bin
Deleting older checkpoint [training_with_callbacks/checkpoint-1000] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 2400
  Batch size = 4
Saving model checkpoint to training_with_callbacks/checkpoint-4000
Configuration saved in training_with_callbacks/checkpoint-4000/config.json
Model weights saved in training_with_callbacks/checkpoint-4000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 2400
  Batch size = 4
Saving model checkpoint to training_with_callbacks/checkpoint-6000
Configuration saved in training_with_callbacks/checkpoint-6000/config.json
Model weights saved in training_with_callbacks/checkpoint-6000/pytorch_model.bin
Deleting older checkpoin

TrainOutput(global_step=29945, training_loss=0.6025905337152945, metrics={'train_runtime': 14120.1167, 'train_samples_per_second': 8.482, 'train_steps_per_second': 2.121, 'total_flos': 3.15128111004672e+16, 'train_loss': 0.6025905337152945, 'epoch': 5.0})

In [55]:
trainer.evaluate(small_test_dataset)

***** Running Evaluation *****
  Num examples = 2400
  Batch size = 4


{'eval_loss': 0.5425449013710022,
 'eval_f1': 0.8672566371681415,
 'eval_runtime': 80.7254,
 'eval_samples_per_second': 29.73,
 'eval_steps_per_second': 7.433,
 'epoch': 5.0}

In [47]:
#install huggingface_hub package to interact with huggingface platform
!pip install huggingface_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [48]:
from huggingface_hub import notebook_login

In [49]:
#login to huggingface
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.huggingface/token
Login successful


In [50]:
#specify the path for the trainde model and tokenizer to huggingface repository
model.save_pretrained("path/to/amharic-hate-speech-detection-mBERT")
tokenizer.save_pretrained("path/to/amharic-hate-speech-detection-mBERT")

Configuration saved in path/to/amharic-hate-speech-detection-mBERT/config.json
Model weights saved in path/to/amharic-hate-speech-detection-mBERT/pytorch_model.bin
tokenizer config file saved in path/to/amharic-hate-speech-detection-mBERT/tokenizer_config.json
Special tokens file saved in path/to/amharic-hate-speech-detection-mBERT/special_tokens_map.json


('path/to/amharic-hate-speech-detection-mBERT/tokenizer_config.json',
 'path/to/amharic-hate-speech-detection-mBERT/special_tokens_map.json',
 'path/to/amharic-hate-speech-detection-mBERT/vocab.txt',
 'path/to/amharic-hate-speech-detection-mBERT/added_tokens.json',
 'path/to/amharic-hate-speech-detection-mBERT/tokenizer.json')

In [56]:
#push the trained model to huggingface repository
model.push_to_hub("amharic-hate-speech-detection-mBERT")

Configuration saved in /tmp/tmpfamfkyyi/config.json
Model weights saved in /tmp/tmpfamfkyyi/pytorch_model.bin
Uploading the following files to amengemeda/amharic-hate-speech-detection-mBERT: pytorch_model.bin,config.json


CommitInfo(commit_url='https://huggingface.co/amengemeda/amharic-hate-speech-detection-mBERT/commit/5eba179aa9faacf7d75237f636daea17f338e964', commit_message='Upload BertForSequenceClassification', commit_description='', oid='5eba179aa9faacf7d75237f636daea17f338e964', pr_url=None, pr_revision=None, pr_num=None)

In [57]:
#push the tokenizer to huggingface repository
tokenizer.push_to_hub("amharic-hate-speech-detection-mBERT")

tokenizer config file saved in /tmp/tmpy24b1ker/tokenizer_config.json
Special tokens file saved in /tmp/tmpy24b1ker/special_tokens_map.json
Uploading the following files to amengemeda/amharic-hate-speech-detection-mBERT: vocab.txt,tokenizer.json,tokenizer_config.json,special_tokens_map.json


CommitInfo(commit_url='https://huggingface.co/amengemeda/amharic-hate-speech-detection-mBERT/commit/1b165d50785009496b0822f597ef8904c1cb14a3', commit_message='Upload tokenizer', commit_description='', oid='1b165d50785009496b0822f597ef8904c1cb14a3', pr_url=None, pr_revision=None, pr_num=None)